# IRS TAXES
The IRS repository: https://www.irs.gov/downloads/irs-soi?page=17 offers a lot of excel files on regions and communities of the whole US.

In [30]:
import os
import pandas as pd

## File download

With a simple wget script I will download the relevant files for Alaska. I could manually find only 2011 to 2022 files, paying attention to the fact that before 2017 they are XLS files and not XLSX. 

I suspect this was the reason for why the dnr file only went back to 2017 - they didn't wget the XLS files, so the XLSX links returned 404 errors.

In [31]:
import wget

original_files_path = '../../data/irs_tax_data/original'
csv_files_path = '../../data/irs_tax_data/csv'
years = list(range(11, 23))

for year in years:
    if year > 16:
        filetype = 'xlsx'
    else:
        filetype = 'xls'
    
    filename = str(year) + "incyak." + filetype
    target_link = "https://www.irs.gov/pub/irs-soi/" + filename
    
    # wget the target file:
    wget.download(target_link, out = original_files_path)    

## Convert all to CSV
Some manipulation for cleanup is needed, but turning all of them into csv's reduses warnings and encoding walkarounds.

In [32]:
csv_files_path = '../../data/irs_tax_data/modified_csvs'

for file in os.listdir(original_files_path):
    print(file)
    dft = pd.read_excel(os.path.join(original_files_path, file), skiprows=3)
    
    csv_file_name = str(file).split('.')[0] + '.csv'
    dft.to_csv(os.path.join(csv_files_path, csv_file_name))

18incyak.xlsx
14incyak (1).xls
15incyak (1).xls
16incyak (1).xls
17incyak (1).xlsx
21incyak (1).xlsx
19incyak (1).xlsx
20incyak.xlsx
15incyak.xls
17incyak.xlsx
11incyak (1).xls
22incyak (1).xlsx
19incyak.xlsx
16incyak.xls
11incyak.xls
13incyak (1).xls
12incyak (1).xls
22incyak.xlsx
18incyak (1).xlsx
12incyak.xls
13incyak.xls
21incyak.xlsx
14incyak.xls
20incyak (1).xlsx


## Cleanup files
The files need some cleanup and streamlining before jointing.

In [33]:
# Define the column containing the county names and income brackets
def add_area_column(df, col_name):
    df["county"] = df[col_name].where(df[col_name].str.contains("Borough", na=False) | df[col_name].str.contains("Census Area", na=False))
    df["county"] = df["county"].ffill().copy()
    df = df.dropna(subset=[col_name])
    df = df.dropna(subset=['county'])
    
    # now I remove the row where the county is specified under size of adjusted income
    df = df[~(df[col_name].str.contains("Borough", na=False) | df[col_name].str.contains("Census Area", na=False))]

    df.reset_index(drop=True, inplace=True)
        
    
    # save
    return df

In [34]:
csv_files_path = '../../data/irs_tax_data/modified_csvs'

for file in os.listdir(csv_files_path):
    file_path = os.path.join(csv_files_path, file)
    
    dft = pd.read_csv(file_path)
    dft = add_area_column(dft, 'Size of adjusted gross income by county')
    
    print(file)
    print(dft.columns)
    
    dft = dft.copy()
    dft['year'] = str(20) + file[0:2] # the files have the year as first 2 characters of the filename
        
    # reorder columns
    columns = dft.columns.tolist()
    first_columns = ['year', 'county']
    other_cols = [col for col in columns if col not in first_columns]
    dft = dft[first_columns + other_cols]
    
    dft.to_csv(file_path)


22incyak.csv
Index(['Unnamed: 0', 'County \nFIPS \ncode [1]',
       'Size of adjusted gross income by county', 'Number of returns [2]',
       'Number of single returns', 'Number of joint returns',
       'Number of head of household returns',
       'Number of electronically filed returns',
       'Number of computer prepared paper returns',
       'Number with paid preparer's signature',
       ...
       'Unnamed: 154', 'Tax due at time of filing [15]', 'Unnamed: 156',
       'Total overpayments', 'Unnamed: 158', 'Overpayments refunded [16]',
       'Unnamed: 160', 'Credited to next year's estimated tax', 'Unnamed: 162',
       'county'],
      dtype='object', length=165)
15incyak (1).csv
Index(['Unnamed: 0', 'County \nFIPS \ncode [1]',
       'Size of adjusted gross income by county', 'Number of returns',
       'Number of single returns', 'Number of joint returns',
       'Number of head of household returns',
       'Number with paid preparer's signature', 'Number of exemptions'

## Merge into a single dataframe
Merge all the individual files from 2011 to 2022 into one dataframe for manipulation and saving

In [35]:
master = pd.DataFrame()

file_list = os.listdir(csv_files_path)
file_list.sort()

for file in file_list:
    file_path = os.path.join(csv_files_path, file)
    
    dft = pd.read_csv(file_path)
    
    master = pd.concat([master, dft], ignore_index=True)

## Merge columns and cleanup
The IRS named the columns something like 'Earned income credit [5]' with the number changing between years, but not corresponding to them.

This creates multiple columns with NaN for some years but filling up the NaNs for others. I unify them for clarity and simplified later manipulation.

In [36]:
col_groups = {}
for col in master.columns:
    stripped_name = col.rsplit('[', 1)[0].strip() if '[' in col else col
    col_groups.setdefault(stripped_name, []).append(col)

# Unify if there is no overlap
# columns with different [numbers] represent different years, there should be no overlap
safe_to_unify = []
for base_name, cols in col_groups.items():
    if len(cols) > 1:
        df_slice = master[cols].notna().sum(axis=1)
        if not (df_slice > 1).any():  # Check if safe
            master[base_name] = master[cols].bfill(axis=1).iloc[:, 0]
            master.drop(columns=cols, inplace=True)
            safe_to_unify.append(base_name)

# Original order
essential_cols = ['year', 'county']
ordered_cols = essential_cols + [col for col in master.columns if col not in essential_cols]
master = master[ordered_cols]

/tmp/ipykernel_2301067/876518976.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  master[base_name] = master[cols].bfill(axis=1).iloc[:, 0]
/tmp/ipykernel_2301067/876518976.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  master[base_name] = master[cols].bfill(axis=1).iloc[:, 0]
/tmp/ipykernel_2301067/876518976.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1

In [37]:
[x for x in master.columns.tolist() if 'leave' in x]

['Qualified sick and family leave credit',
 'Qualified sick and family leave credit \nfor leave taken beore April 1, 2021',
 'Qualified sick and family leave credit \nfor leave taken after March 31, 2021']

## Compute The tax burden

I define the Tax burdens as such:
- Total Tax Burden 
\[
\text{Total Tax Burden} = \frac{\text{Total Tax Liability}}{\text{Adjusted Gross Income (AGI)}}
\]
- Federal Tax Burden = 
\[
\text{Federal Tax Burden} = \frac{\text{Total Tax Liability} - \text{Local Taxes Paid}}{\text{Adjusted Gross Income (AGI)}}
\]
- Local Tax Burden = 
\[
\text{Local Tax Burden} = \frac{\text{State and Local Income Taxes + State and Local General Sales Taxes + Real Estate Taxes + Personal Property Taxes}}{\text{Adjusted Gross Income (AGI)}}
\]

In [38]:
import numpy as np

# Combine AGI columns
agi_columns = [col for col in master.columns if "Adjusted gross income (AGI)" in col]
master["AGI_clean"] = master[agi_columns].bfill(axis=1).iloc[:, 0]

# Preserve original NaNs and zeros to avoid misrepresentation
master["AGI_clean"] = np.where(master[agi_columns].isna().all(axis=1), np.nan, master["AGI_clean"])
master["AGI_clean"] = np.where(master[agi_columns].eq(0).all(axis=1), 0, master["AGI_clean"])

# Not all Alaskan Boroughs apply local taxes such as income or personal property taxes
# they thus come up as NaN but it's because they are 0, so I fill them with 0
master["Local_taxes_paid"] = (
    master["State and local income taxes"].fillna(0) +
    master["State and local general sales taxes"].fillna(0) +
    master["Real estate taxes"].fillna(0) +
    master["Personal property taxes"].fillna(0)
)


master["total_tax_burden"] = master["Total tax liability"] / master["AGI_clean"]
master["local_tax_burden"] = master["Local_taxes_paid"] / master["AGI_clean"]
master["federal_tax_burden"] = (
    master["Total tax liability"] - master["Local_taxes_paid"]
) / master["AGI_clean"]

# Set burden to NaN where AGI is zero or NaN to avoid misleading ratios
master.loc[
    master["AGI_clean"].isna() | (master["AGI_clean"] == 0), 
    ["total_tax_burden", "local_tax_burden", "federal_tax_burden"]
] = np.nan

master['child_and_other_dependent_credit'] = master['Child and other dependent credit']
master['social_security_benefits'] = master['Social Security benefits in AGI']
master['sick_and_family_leave_credit'] = master['Qualified sick and family leave credit']

/tmp/ipykernel_2301067/2321295100.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  master["AGI_clean"] = master[agi_columns].bfill(axis=1).iloc[:, 0]
/tmp/ipykernel_2301067/2321295100.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  master["Local_taxes_paid"] = (
/tmp/ipykernel_2301067/2321295100.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To ge

In [39]:
master.columns.tolist()

['year',
 'county',
 'Unnamed: 0.1',
 'Unnamed: 0',
 'County \nFIPS \ncode [1]',
 'Size of adjusted gross income by county',
 'Number of joint returns',
 "Number with paid preparer's signature",
 'Number of exemptions',
 'Number of dependents',
 'Salaries and wages in AGI',
 'Unnamed: 9',
 'Taxable interest',
 'Unnamed: 11',
 'Ordinary dividends',
 'Unnamed: 13',
 'Unnamed: 15',
 'Business or profession\nnet income (less loss)',
 'Unnamed: 17',
 'Number of \nfarm returns',
 'Net capital gain (less loss) in AGI',
 'Unnamed: 20',
 'Taxable individual retirement\narrangement distributions',
 'Unnamed: 22',
 'Pensions and annuities in AGI',
 'Unnamed: 24',
 'Unnamed: 26',
 'Social Security benefits in AGI',
 'Unnamed: 28',
 'Self-employment retirement plans',
 'Unnamed: 30',
 'Total itemized deductions',
 'Unnamed: 32',
 'State and local income taxes',
 'Unnamed: 34',
 'State and local general sales taxes',
 'Unnamed: 36',
 'Real estate taxes',
 'Unnamed: 38',
 'Taxes paid',
 'Unnamed: 40'

## Compute inequality measure (GINI)
Given the division of the population into income brakets, I compute the GINI coefficient as measure for inequality:
\[
G = 1 - \sum_{i=1}^{n}(X_i - X_{i-1})(Y_i + Y_{i-1})
\]

Where:

- \(X_i\) is the cumulative share of the population up to the income bracket \(i\).
- \(Y_i\) is the cumulative share of income up to the income bracket \(i\).

In [40]:
# compute the total number of returns filling NaNs as zeroes
master["Number_of_returns"] = (
    master["Number of single returns"].fillna(0) +
    master["Number of joint returns"].fillna(0) +
    master["Number of head of household returns"].fillna(0)
)

# Explicitly map income brackets to midpoints
income_bracket_to_midpoint = {
    "Under $1": 500,
    "$1 under $10,000": 5000,
    "$10,000 under $25,000": 17500,
    "$25,000 under $50,000": 37500,
    "$50,000 under $75,000": 62500,
    "$75,000 under $100,000": 87500,
    "$100,000 under $200,000": 150000,
    "$200,000 or more": 250000
}

/tmp/ipykernel_2301067/1056130966.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  master["Number_of_returns"] = (


In [41]:
def gini_coefficient(incomes, weights):    
    # Sort incomes and weights
    sorted_indices = np.argsort(incomes)
    incomes = incomes[sorted_indices]
    weights = weights[sorted_indices]

    cum_weights = np.cumsum(weights)
    cum_income = np.cumsum(incomes * weights)

    # Normalize cumulative sums
    cum_weights_normalized = cum_weights / cum_weights[-1]
    cum_income_normalized = cum_income / cum_income[-1]

    # Calculate Gini using trapezoidal approximation
    gini = 1 - 2 * np.trapz(cum_income_normalized, cum_weights_normalized)
    return gini

# Compute Gini by group
def compute_gini_by_group(df):
    bracket_names = df["Size of adjusted gross income by county"].values
    counts = df["Number_of_returns"].values
    
    # Map income brackets to midpoints using the explicit map created above
    midpoints = np.array([income_bracket_to_midpoint.get(bracket, np.nan) for bracket in bracket_names])

    # Drop rows with NaN midpoints if any
    valid = ~np.isnan(midpoints)
    midpoints = midpoints[valid]
    counts = counts[valid]

    if counts.sum() == 0:
        return np.nan
    
    return gini_coefficient(midpoints, counts)

# Apply the function
gini_df = master.groupby(['county', 'year'], group_keys=False).apply(compute_gini_by_group).reset_index()
gini_df.columns = ['county', 'year', 'Gini']

# Merge Gini back to master
master = master.merge(gini_df, on=['county', 'year'], how='left')

# Save as single file in the custom directory

In [42]:
# Save to the custom data directory
csv_files_path = '../../data/irs_tax_data/modified_csvs'

custom_data_directory = '../../data/custom_data'
irs_taxes_merged_path = os.path.join(custom_data_directory, 'irs_taxes_custom.csv')

master.to_csv(irs_taxes_merged_path)